In [3]:
import pandas as pd
import nltk
import re

from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

Dataset

In [4]:
df = pd.read_csv("Dataset/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [5]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


In [6]:
df['essay'][0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [7]:
temp.head(1)

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [8]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
X_train.shape

(9083, 3)

Pre-processing

In [11]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [12]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [13]:
len(train_sents)

115924

In [14]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

In [15]:
train_sents

[['It',
  'first',
  'day',
  'high',
  'school',
  'gut',
  'full',
  'butterflies',
  'make',
  'want',
  'run',
  'bathrooms',
  'hide',
  'world'],
 ['I',
  'CAPS',
  'friends',
  'yet',
  'I',
  'shy',
  'go',
  'join',
  'groups',
  'I',
  'walked',
  'classes',
  'alone'],
 ['Students',
  'would',
  'get',
  'face',
  'wanting',
  'know',
  'school',
  'I',
  'came',
  'wanted',
  'know'],
 ['I', 'remember', 'wanting', 'left', 'alone', 'hidden', 'bubble', 'space'],
 ['Girls',
  'would',
  'ask',
  'CAPS',
  'I',
  'upset',
  'since',
  'I',
  'smiling',
  'fun',
  'getting',
  'know',
  'new',
  'classes'],
 ['Lunch', 'much', 'crowded', 'old', 'school'],
 ['Students',
  'would',
  'sit',
  'hallways',
  'eating',
  'lunch',
  'crowd',
  'around',
  'one',
  'big',
  'table',
  'chattering',
  'loudly',
  'one',
  'another'],
 ['I', 'chewed', 'lip', 'looking', 'spot', 'occupied', 'students'],
 ['I',
  'found',
  'table',
  'near',
  'front',
  'lunch',
  'room',
  'empty',
  'pla

word2vec model

In [17]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

<hr>

Converting training and testing sentences to vectors using word2vec

In [18]:
import numpy as np

In [ ]:
def makeVec(words, model, num_features):
    vec=[]
    # Average word2vec vector generation
    
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs=[]
    for i in essays: # iterating all sentences in essay
        essay_vecs.append( makeVec(i, model, num_features) )
        c+=1
    return essay_vecs